# Advanced Assignment: Data Visualization Dashboard

In this assignment, we'll create a comprehensive data visualization dashboard using Matplotlib to analyze and present insights from the World Happiness Report dataset.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.animation import FuncAnimation

%matplotlib inline
plt.style.use('default')

## 1. Data Preparation

In [2]:
# Load the data
# Note: You need to download the World Happiness Report dataset and adjust the file path accordingly
df = pd.read_csv('world_happiness_report.csv')

# Data cleaning and preprocessing
df['Year'] = pd.to_datetime(df['Year'], format='%Y')
df = df.sort_values('Year')

# Display the first few rows and data info
print(df.head())
print(df.info())

FileNotFoundError: [Errno 2] No such file or directory: 'world_happiness_report.csv'

## 2. Create the Dashboard

In [ ]:
# Set up the dashboard layout
fig = plt.figure(figsize=(20, 30))
gs = GridSpec(3, 2, figure=fig)

# Color scheme
colors = plt.cm.viridis(np.linspace(0, 1, 10))

# a. Line Plot
ax1 = fig.add_subplot(gs[0, 0])
yearly_avg = df.groupby('Year')['Life Ladder'].mean()
yearly_std = df.groupby('Year')['Life Ladder'].std()

ax1.errorbar(yearly_avg.index, yearly_avg.values, yerr=yearly_std.values, color=colors[0], ecolor=colors[1], capsize=5)
ax1.set_title('Average Happiness Score Over Years', fontsize=16)
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Average Happiness Score', fontsize=12)

# b. Bar Chart
ax2 = fig.add_subplot(gs[0, 1])
latest_year = df['Year'].max()
top_10 = df[df['Year'] == latest_year].nlargest(10, 'Life Ladder')

cmap = LinearSegmentedColormap.from_list("", ["#FF9999","#99FF99"])
colors = cmap(np.linspace(0, 1, len(top_10)))

bars = ax2.bar(top_10['Country name'], top_10['Life Ladder'], color=colors)
ax2.set_title(f'Top 10 Happiest Countries in {latest_year.year}', fontsize=16)
ax2.set_xlabel('Country', fontsize=12)
ax2.set_ylabel('Happiness Score', fontsize=12)
ax2.tick_params(axis='x', rotation=45)

for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}', 
             ha='center', va='bottom')

# c. Scatter Plot
ax3 = fig.add_subplot(gs[1, 0])
latest_data = df[df['Year'] == latest_year]

for continent in latest_data['Regional indicator'].unique():
    continent_data = latest_data[latest_data['Regional indicator'] == continent]
    ax3.scatter(continent_data['Log GDP per capita'], continent_data['Life Ladder'], 
                label=continent, alpha=0.6)

ax3.set_title('GDP per capita vs. Happiness Score', fontsize=16)
ax3.set_xlabel('Log GDP per capita', fontsize=12)
ax3.set_ylabel('Happiness Score', fontsize=12)
ax3.legend(fontsize=8, loc='lower right')

# Add best-fit line
z = np.polyfit(latest_data['Log GDP per capita'], latest_data['Life Ladder'], 1)
p = np.poly1d(z)
ax3.plot(latest_data['Log GDP per capita'], p(latest_data['Log GDP per capita']), "r--", alpha=0.8)

# d. Box Plot
ax4 = fig.add_subplot(gs[1, 1])
df.boxplot(column=['Life Ladder'], by='Regional indicator', ax=ax4)
ax4.set_title('Distribution of Happiness Scores Across Continents', fontsize=16)
ax4.set_xlabel('Continent', fontsize=12)
ax4.set_ylabel('Happiness Score', fontsize=12)
ax4.tick_params(axis='x', rotation=90)

# e. Heatmap
ax5 = fig.add_subplot(gs[2, 0])
factors = ['Life Ladder', 'Log GDP per capita', 'Social support', 'Healthy life expectancy at birth',
           'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']
corr = df[factors].corr()

im = ax5.imshow(corr, cmap='coolwarm')
ax5.set_xticks(np.arange(len(factors)))
ax5.set_yticks(np.arange(len(factors)))
ax5.set_xticklabels(factors, rotation=45, ha='right')
ax5.set_yticklabels(factors)
for i in range(len(factors)):
    for j in range(len(factors)):
        text = ax5.text(j, i, f"{corr.iloc[i, j]:.2f}",
                        ha="center", va="center", color="black")
ax5.set_title('Correlation Between Factors', fontsize=16)
plt.colorbar(im)

# f. Pie Chart
ax6 = fig.add_subplot(gs[2, 1])
top_country = top_10.iloc[0]
factors = ['Log GDP per capita', 'Social support', 'Healthy life expectancy at birth',
           'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']
values = top_country[factors]

ax6.pie(values, labels=factors, autopct='%1.1f%%', startangle=90, colors=colors)
ax6.set_title(f'Factor Importance for {top_country["Country name"]}', fontsize=16)

plt.tight_layout()
plt.savefig('world_happiness_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. Interactivity

In [ ]:
def on_click(event):
    if event.inaxes == ax2:
        for i, bar in enumerate(bars):
            if bar.contains(event)[0]:
                country = top_10.iloc[i]['Country name']
                score = top_10.iloc[i]['Life Ladder']
                print(f"{country}: Happiness Score = {score:.2f}")

fig.canvas.mpl_connect('button_press_event', on_click)

# Note: This interactivity will only work in an interactive environment like Jupyter Notebook

## 4. Bonus Challenges

### 4.1 Add annotations to highlight key insights

In [ ]:
ax1.annotate('Global financial crisis', xy=(pd.to_datetime('2008-01-01'), 5.4),
             xytext=(pd.to_datetime('2006-01-01'), 5.2),
             arrowprops=dict(facecolor='black', shrink=0.05))

plt.show()

### 4.2 Create an animated plot

In [ ]:
def animate(year):
    data = df[df['Year'] == pd.to_datetime(year, format='%Y')]
    ax.clear()
    for country in ['United States', 'China', 'India', 'Japan', 'Germany']:
        country_data = data[data['Country name'] == country]
        ax.scatter(country_data['Log GDP per capita'], country_data['Life Ladder'], label=country)
    ax.set_xlim(6, 12)
    ax.set_ylim(2, 8)
    ax.set_title(f'Happiness vs GDP ({year})')
    ax.set_xlabel('Log GDP per capita')
    ax.set_ylabel('Happiness Score')
    ax.legend()

fig, ax = plt.subplots(figsize=(10, 6))
ani = FuncAnimation(fig, animate, frames=range(2005, 2021), interval=500, repeat=False)
plt.show()

### 4.3 Implement a custom colormap

In [ ]:
happiness_cmap = LinearSegmentedColormap.from_list("happiness", ['#FF9999', '#FFFF99', '#99FF99'], N=100)
plt.figure(figsize=(15, 10))
plt.imshow(df.pivot(index='Country name', columns='Year', values='Life Ladder'), 
           cmap=happiness_cmap, aspect='auto')
plt.colorbar(label='Happiness Score')
plt.title('Happiness Scores Over Time by Country')
plt.xlabel('Year')
plt.ylabel('Country')
plt.show()

## 5. Conclusion

This dashboard provides a comprehensive visualization of the World Happiness Report data. It includes various types of plots to showcase different aspects of the data, from trends over time to correlations between factors. The interactive elements and bonus visualizations offer additional insights into the complex relationships within the dataset.